#### Data PreProcess

In [1]:
import pandas as pd
import os
print(os.getcwd())

d:\Master\PracticalMachineLearning\Proiect1


In [2]:
df_train = pd.read_json(f'{os.getcwd()}/train.json', lines=False)
df_valid = pd.read_json(f'{os.getcwd()}/validation.json', lines=False)
df_test = pd.read_json(f'{os.getcwd()}/test.json', lines=False)

print(len(df_train))
df = pd.concat([df_train, df_valid, df_test])
df

58114


,sentence1,sentence2,label,guid
0,Primul taragotist român a fost Nicolae Luță Io...,"Colegiul de arhitectură, artă și planificare (...",3.0,7cec5ac4-c115-4976-8d2a-9badfe9b63b9
1,Lupta revoluționarilor este condusă de Avram I...,Schiul nordic face parte din programul olimpic...,3.0,bc2fa29f-cf22-4a7c-8b55-9b1ed019f6ac
2,Locuitorii liberi au devenit „''iobagiones cas...,"În anii 1960, ea a apărut în drame realizate l...",3.0,8547b1ef-7bfe-43a9-aedf-bad0c0fbc049
3,În anul 2002 are loc lansarea în domeniul turi...,Se lansează primul hotel al grupului în otopen...,2.0,0ad1ce19-7aa9-4ddd-b8d6-822072a723b0
4,"Zillich a mijlocit, prin revista ''Klingsor'',...","Au apărut lucrări ale lui ion luca caragiale, ...",2.0,50c44ffa-b0c1-4d98-bc6c-3bbf95f50896
...,...,...,...,...
2995,*Totemurile misterioase (Mysetry totems) - sun...,Operațiunea aliată a redus capul de pod german...,NaN,74ca470e-d28a-42dc-bfcc-7f1e57ff042b
2996,"Univers, 1986\n* ''Ore de dans pentru vârstnic...",Alexandru Ivanovici Tolstoi (1770 — 1857) - ve...,NaN,fc0b744e-c69c-4e8f-9ec8-1495e7a3725c
2997,"Vena hemiazygos și vena accesorie hemiazygos, ...",Vena azygos servește la drenarea majorității v...,NaN,538041fd-c1b7-481a-8a40-272b851ff3c7
2998,"În fapt, arestările din cadrul grupului încep ...",Acesta își propunea doar îngenunchierea credin...,NaN,9b8dcd8c-fef9-4a2d-a170-b972461103d0


In [3]:
s1 = df['sentence1'].tolist()
s2 = df['sentence2'].to_list()

sentences = [(f'{df["sentence1"].iloc[index]} \n {df["sentence2"].iloc[index]}') for index in range(len(df['sentence1'])) ]
reversed_sentences = [(f'{df["sentence2"].iloc[index]}\n{df["sentence1"].iloc[index]}') for index in range(len(df['sentence1'])) ]
labels = df['label'].to_list()
guids = df['guid'].to_list()
sentences

['Primul taragotist român a fost Nicolae Luță Ioviță, originar din Banat. \n Colegiul de arhitectură, artă și planificare (College of Architecture, Art and Planning) are asigurate studiouri și spații pentru seminare pentru studenți și cadre didactice  pe strada West 17th, lângă Union Square.',
 'Lupta revoluționarilor este condusă de Avram Iancu, Ioan Ciurileanu, frații Ioan Rațiu (viitorul memorandist) și Partenie Rațiu, verii lor Ioan Rațiu "Tribunul" și Nicolae Mureșan, viitor martir alături de Alexandru Arpadi. \n Schiul nordic face parte din programul olimpic începând cu Jocurile Olimpice de iarnă din 1924.',
 "Locuitorii liberi au devenit „''iobagiones castri''” (iobagi ai cetății) cu certe obligații militare în apărarea marginilor feudei de la Lotru, Turnu Roșu și Tălmaciu. \n În anii 1960, ea a apărut în drame realizate la televiziune și în trei filme, ultimul fiind „The Pleasure Seekers”, în 1964 și la televiziune, în 1980, ultimul fiind „Scruples”.",
 'În anul 2002 are loc la

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(**{
    'min_df': 1,
    'max_features': None,
    'strip_accents': 'unicode',
    'analyzer': 'word',
    'token_pattern': r'\b[^\d\W]+\b',
    'ngram_range': (1, 3),
    'use_idf': True,
    'smooth_idf': True,
    'sublinear_tf': True,
    'norm': 'l2'
})

In [5]:
train_data_tf_idf = vectorizer.fit_transform(sentences[:58114])
valid_data_tf_idf = vectorizer.transform(sentences[58114:61173])
test_data_tf_idf = vectorizer.transform(sentences[61173:])

In [7]:
train_data_tf_idf.shape

(58114, 3335464)

###### New TF-IDF

#### Data Normalization

In [7]:
import numpy as np
from sklearn.preprocessing import normalize
normalized_train_data = normalize(train_data_tf_idf)
normalized_train_data
normalized_valid_data = normalize(valid_data_tf_idf)
normalized_test_data = normalize(test_data_tf_idf)


In [ ]:
import contractions
import re
import string
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('romanian'))
lemmatizer = WordNetLemmatizer()

def preProcess(data, text:str):
  data[text] = data[text].apply(lambda x:contractions.fix(x))
  data[text] = data[text].apply(lambda x: x.lower())
  data[text] = data[text].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))

  def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
  data[text] = data[text].apply(lambda x: remove_stopwords(x))

  def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
  data[text] = data[text].apply(lambda x: lemmatize_words(x))

  data[text] = data[text].apply(lambda x: re.sub(' +', ' ', x))
  return data[text]

### WordEmbedings


In [44]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('romanian'))
MAX_WORDS = 0
BIG_LIST=[]

def remove_stopwords(text:str):
    global MAX_WORDS
    global BIG_LIST
    list_of_words = [word for word in text.lower().split() if word not in stopwords]
    if len(list_of_words) > MAX_WORDS:
        MAX_WORDS = len(list_of_words)
    return (list_of_words)


s1 = df["sentence1"]
s1 = pd.DataFrame(s1)
s1 = s1.rename(columns={"sentence1": "sentence"})

s2 = df["sentence2"]
s2 = pd.DataFrame(s2)
s2 = s2.rename(columns={"sentence2": "sentence"})

st = pd.concat([s1,s2])
#clean_sentences = st["sentence"].apply(lambda x: remove_stopwords(x))
st_df = pd.DataFrame(sentences)
st_df[0]
clean_sentences = st_df[0].apply(lambda x: remove_stopwords(x))
clean_sentences.iloc[1]

from gensim.models import Word2Vec
embedings_model = Word2Vec(clean_sentences, vector_size=200, workers=4)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dragos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
BIG_LIST

[]

In [42]:
MAX_WORDS

1534

In [49]:
embedings_model.wv.most_similar(['comitatul'], topn=3)

[('prahova', 0.9627609252929688),
 ('mureș,', 0.9469833374023438),
 ('bihor', 0.9444875121116638)]

In [106]:
clean_sentences.iloc[39325]

['localitatea',
 'făcea',
 'parte',
 'perioada',
 'comitatul',
 'dăbâca,',
 'moșia',
 'fiind',
 'împărțită',
 'între',
 'mulți',
 'proprietari.']

In [52]:
import numpy as np
def sentence_to_vector(sentence:str, sentence_length:int):
    words = sentence.split()
    vector=[]
    vector_length = len(words)-1 if len(words) < sentence_length else sentence_length-1
    for w in words[:vector_length]:
        try:
            vector.append(embedings_model.wv[w.lower()])
        except Exception:
            pass
    last_pieces = sentence_length - len(vector)
    for i in range(last_pieces):
        vector.append(np.zeros(200,))
        
    return np.asarray(vector).flatten()

In [53]:
ss = sentence_to_vector(sentences[0],1500)
ss.shape

(300000,)

In [57]:
train_vec = np.asarray([sentence_to_vector(s,1500) for s in sentences[:58114]])
valid_vec = np.asarray([sentence_to_vector(s,1500) for s in sentences[58114:61173]])
test_vec = np.asarray([sentence_to_vector(s,1500) for s in sentences[61173:]])
train_vec[0].shape

MemoryError: Unable to allocate 2.29 MiB for an array with shape (300000,) and data type float64

In [131]:
train_vec.shape

(58114, 2500)

In [132]:
from sklearn.decomposition import PCA
pca_model = PCA(n_components=50)
pca_model.fit(train_vec)

PCA(n_components=50)

In [133]:
train_vec = pca_model.fit_transform(train_vec)
valid_vec = pca_model.fit_transform(valid_vec)
test_vec = pca_model.fit_transform(test_vec)

In [134]:
train_y = df.iloc[:58114]['label']
valid_y = df.iloc[58114:61173]['label']

In [135]:
from sklearn.svm import LinearSVC, SVC, NuSVC
model = LinearSVC(C=10)
model.fit(train_vec, train_y)
preds = model.predict(valid_vec)

c:\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [136]:
from sklearn.metrics import balanced_accuracy_score #ACC
from sklearn.metrics import f1_score #MCC
# https://scikit-learn.org/stable/modules/model_evaluation.html#matthews-corrcoef
from sklearn.metrics import multilabel_confusion_matrix
balanced_accuracy = round(100*balanced_accuracy_score(valid_y, preds), 2)
f1 = round(f1_score(valid_y, preds, average='macro'), 2)
print(f1)
print(balanced_accuracy)

0.3
30.98


In [121]:
test_preds = model.predict(test_vec)
test_guids = df_test['guid']

In [122]:
import csv
fields = ['guid', 'label']
rows = [ [test_guids.iloc[index], int(test_preds[index])] for index in range(len(test_guids))]

filename = "Word_embedings 40x40.csv"

with open(filename, 'w') as csvfile:   
    csvwriter = csv.writer(csvfile)  
        
    csvwriter.writerow(fields)  
         
    csvwriter.writerows(rows) 

#### Feature engineering

In [7]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import pearsonr, spearmanr

nltk.download('stopwords')
stopwords = set(stopwords.words('romanian'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stopwords])

def get_clean_sentences(pair:str):
    pair = pair.split(sep='\n')
    s1 = remove_stopwords(pair[0])
    s2 = remove_stopwords(pair[1])
    return s1,s2

def get_jaccard_similarity(sentence1:str, sentence2:str):
    words1 = set(sentence1.lower().split())
    words2 = set(sentence2.lower().split())
    intersection_size = len(words1.intersection(words2))
    union_size = len(words1.union(words2))
    jaccard_similarity = intersection_size / union_size if union_size != 0 else 0.0
    return jaccard_similarity

def get_cosine_similarity(s1:str, s2:str):
    vectorizer = TfidfVectorizer()
    try:
        tf_idf = vectorizer.fit_transform([s1, s2])
        tf_idf1 = tf_idf[0]
        tf_idf2 = tf_idf[1]
        cos = cosine_similarity(tf_idf1, tf_idf2)
        return cos[0][0]
    except ValueError:
        return 0

def cosine_similarity_ngrams(sentence1:str, sentence2:str, n):
    count_vectorizer = CountVectorizer(ngram_range=(n, n))
    try:
        ngrams_matrix = count_vectorizer.fit_transform([sentence1, sentence2])
        cosine_sim = cosine_similarity(ngrams_matrix)[0, 1]
        return cosine_sim
    except ValueError:
        return 0
    
def get_pearson_correlation(sentence1:str, sentence2:str):
    words1 = set(sentence1.lower().split())
    words2 = set(sentence2.lower().split())
    return pearsonr(words1, words2)[0]

def get_spearman_correlation(sentence1:str, sentence2:str):
    return spearmanr(sentence1, sentence2)[0]

s1,s2 = get_clean_sentences(sentences[20])
cos = get_cosine_similarity(s1, s2)
ps = get_pearson_correlation(s1, s2)
sp = get_spearman_correlation(s1, s2)
print(f"{cos} {ps} {sp}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dragos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ValueError: x and y must have the same length.

In [49]:
s1,s2 = get_clean_sentences(sentences[0])
cos = get_cosine_similarity(df_train['sentence1'].iloc[0], df_train['sentence2'].iloc[0])
print(cos)

[[0.]]


In [46]:
def get_features(sentences:str):
    features = []
    for pair in sentences:
        s1,s2 = get_clean_sentences(pair)
        jac_similarity = get_jaccard_similarity(s1, s2)
        cos_similarity = get_cosine_similarity(s1, s2)
        unigram_cos_similarity = cosine_similarity_ngrams(s1, s2, 1)
        bigram_cos_similarity = cosine_similarity_ngrams(s1, s2, 2)
        trigram_cos_similarity = cosine_similarity_ngrams(s1, s2, 3)
        features.append([jac_similarity, unigram_cos_similarity, bigram_cos_similarity, trigram_cos_similarity])

    return features

train_features = get_features(sentences[:58114])
valid_features = get_features(sentences[58114:61173])


KeyboardInterrupt: 

In [35]:
test_features = get_features(sentences[61173:])

In [36]:
np.array(train_features)


array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.02857143, 0.08753762, 0.        , 0.        ],
       ...,
       [0.        , 0.        , 0.        , 0.        ],
       [0.03448276, 0.12038585, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [89]:
vectorizer = TfidfVectorizer(stop_words=None)
train_data_tf_idf = vectorizer.fit_transform(train_features)
valid_data_tf_idf = vectorizer.transform(valid_features)
test_data_tf_idf = vectorizer.transform(test_features)

### SVM

#### Try reversed Sentences

In [81]:
sentences[0]

'Primul taragotist român a fost Nicolae Luță Ioviță, originar din Banat. \n Colegiul de arhitectură, artă și planificare (College of Architecture, Art and Planning) are asigurate studiouri și spații pentru seminare pentru studenți și cadre didactice  pe strada West 17th, lângă Union Square.'

In [13]:
sentences = sentences[:58114] + reversed_sentences[:58114]

train_data_tf_idf = vectorizer.fit_transform(sentences)
valid_data_tf_idf = vectorizer.transform(sentences[58114:61173])
test_data_tf_idf = vectorizer.transform(sentences[61173:])

In [20]:
train_y = labels[:58114] + labels[:58114]
valid_y = df.iloc[58114:61173]['label']

##### Normalized Data tryout

In [44]:
from sklearn.svm import LinearSVC, SVC
model = LinearSVC(C=10)
train_y = df.iloc[:58114]['label']
valid_y = df.iloc[58114:61173]['label']

In [45]:
model.fit(normalized_train_data, train_y)
preds = model.predict(normalized_valid_data)

In [42]:
from sklearn.metrics import balanced_accuracy_score #ACC
from sklearn.metrics import f1_score #MCC
# https://scikit-learn.org/stable/modules/model_evaluation.html#matthews-corrcoef
from sklearn.metrics import multilabel_confusion_matrix

In [50]:
balanced_accuracy = round(100*balanced_accuracy_score(valid_y, preds), 2)
f1 = round(f1_score(valid_y, preds, average='weighted'), 2)
print(f1)
print(balanced_accuracy)

0.66
38.7


In [48]:
test_preds = model.predict(normalized_test_data)
test_guids = df_test['guid']

In [49]:
import csv
fields = ['guid', 'label']
rows = [ [test_guids.iloc[index], int(test_preds[index])] for index in range(len(test_guids))]

filename = "Normalized_Tf_Idf_SVC_submission.csv"

with open(filename, 'w') as csvfile:   
    csvwriter = csv.writer(csvfile)  
        
    csvwriter.writerow(fields)  
         
    csvwriter.writerows(rows) 

### Basic Approach

In [72]:
from sklearn.svm import LinearSVC, SVC, NuSVC

In [77]:
model = SVC(C=10, kernel='rbf')
# model = NuSVC(nu=1)

In [78]:
train_y = df.iloc[:58114]['label']
valid_y = df.iloc[58114:61173]['label']

In [150]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 0.3, 0.5, 0.7, 0.9, 1, 3, 5, 7, 10]}
grid_search = GridSearchCV(LinearSVC(), param_grid, cv=5)
grid_search.fit(train_data_tf_idf, train_y)

print("Best Parameters:", grid_search.best_params_)
model = grid_search.best_estimator_

In [79]:
model.fit(train_data_tf_idf, train_y)
preds = model.predict(valid_data_tf_idf)

In [ ]:
from sklearn.metrics import balanced_accuracy_score #ACC
from sklearn.metrics import f1_score #F1
from sklearn.metrics import multilabel_confusion_matrix
balanced_accuracy = round(100*balanced_accuracy_score(valid_y, preds), 2)
f1 = round(f1_score(valid_y, preds, average='macro'), 2)
print(f1)
print(balanced_accuracy)

In [16]:
preds

array([3., 2., 2., ..., 2., 2., 2.])

In [17]:
balanced_accuracy

38.82

In [19]:
test_preds = model.predict(test_data_tf_idf)
test_guids = df_test['guid']

In [3]:
import csv
fields = ['guid', 'label']
rows = [ [test_guids.iloc[index], int(test_preds[index])] for index in range(len(test_guids))]

filename = "FeatureEngineering_submission.csv"

with open(filename, 'w') as csvfile:   
    csvwriter = csv.writer(csvfile)  
        
    csvwriter.writerow(fields)  
         
    csvwriter.writerows(rows) 

NameError: name 'test_guids' is not defined

### Bayes

In [37]:
from sklearn.naive_bayes import ComplementNB, BernoulliNB, CategoricalNB, GaussianNB, MultinomialNB
model = MultinomialNB(alpha=1e0)
model.fit(train_data_tf_idf, train_y)
preds = model.predict(valid_data_tf_idf)

In [39]:
balanced_accuracy = round(100*balanced_accuracy_score(valid_y, preds), 2)
f1 = round(f1_score(valid_y, preds, average='weighted'), 2)
print(f1)
print(balanced_accuracy)

0.67
35.89


In [40]:
test_preds = model.predict(test_data_tf_idf)
test_guids = df_test['guid']

In [41]:
import csv
fields = ['guid', 'label']
rows = [ [test_guids.iloc[index], int(test_preds[index])] for index in range(len(test_guids))]

filename = "MultinomialNB_submission.csv"

with open(filename, 'w') as csvfile:   
    csvwriter = csv.writer(csvfile)  
        
    csvwriter.writerow(fields)  
         
    csvwriter.writerows(rows) 

### XGBoost

In [64]:
import xgboost as xgb

In [9]:
train_y = df.iloc[:58114]['label']
valid_y = df.iloc[58114:61173]['label']
#model = xgb.XGBClassifier(booster="gbtree", n_estimators = 2000, learning_rate=0.1, num_class=4, objective='multi:softmax')

In [66]:
model.fit(train_data_tf_idf, train_y)
preds = model.predict(valid_data_tf_idf)

KeyboardInterrupt: 

In [ ]:
balanced_accuracy = round(100*balanced_accuracy_score(valid_y, preds), 2)
f1 = round(f1_score(valid_y, preds, average='macro'), 2)
print(f1)
print(balanced_accuracy)

0.37
37.39


In [ ]:
test_preds = model.predict(test_data_tf_idf)
test_guids = df_test['guid']

In [ ]:
import csv
fields = ['guid', 'label']
rows = [ [test_guids.iloc[index], int(test_preds[index])] for index in range(len(test_guids))]

filename = "XGBoost_submission_200est.csv"

with open(filename, 'w') as csvfile:   
    csvwriter = csv.writer(csvfile)  
        
    csvwriter.writerow(fields)  
         
    csvwriter.writerows(rows) 

### RandomForest

In [137]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [138]:
model.fit(train_data_tf_idf, train_y)
preds = model.predict(valid_data_tf_idf)

KeyboardInterrupt: 

In [ ]:
balanced_accuracy = round(100*balanced_accuracy_score(valid_y, preds), 2)
f1 = round(f1_score(valid_y, preds, average='macro'), 2)
print(f1)
print(balanced_accuracy)

In [ ]:
test_preds = model.predict(test_data_tf_idf)
test_guids = df_test['guid']

In [ ]:
import csv
fields = ['guid', 'label']
rows = [ [test_guids.iloc[index], int(test_preds[index])] for index in range(len(test_guids))]

filename = "RandomForest_submission.csv"

with open(filename, 'w') as csvfile:   
    csvwriter = csv.writer(csvfile)  
        
    csvwriter.writerow(fields)  
         
    csvwriter.writerows(rows) 

### DL

In [38]:
import tensorflow
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

#The number of predictive features is stored in n_cols.
n_cols = 2

predictors = df[:58114].drop(["guid", "label"], axis = 1).values
target = to_categorical(df["label"][:58114].to_list())

from sklearn.preprocessing import OneHotEncoder

# Assuming 'predictors' contains your features, and 'target' contains your labels
encoder = OneHotEncoder(sparse=False, categories=[range(4)])
target_encoded = encoder.fit_transform(target.reshape(-1, 1))

target_encoded




C:\Users\Dragos\AppData\Local\Temp\ipykernel_22992\1508458318.py:11: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  target = to_categorical(df["label"][:58114].to_list())


array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

In [14]:
train_y = df.iloc[:58114]['label']
valid_y = df.iloc[58114:61173]['label']

In [15]:

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(train_y)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
train_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(train_onehot_encoded)

integer_encoded = label_encoder.fit_transform(valid_y)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
valid_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(valid_onehot_encoded)

[3 3 3 ... 3 2 3]
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


In [6]:

s1_train_data_tf_idf = vectorizer.fit_transform(s1[:58114])
s1_valid_data_tf_idf = vectorizer.transform(s1[58114:61173])
s1_test_data_tf_idf = vectorizer.transform(s1[61173:])
s2_train_data_tf_idf = vectorizer.transform(s2[:58114])
s2_valid_data_tf_idf = vectorizer.transform(s2[58114:61173])
s2_test_data_tf_idf = vectorizer.transform(s2[61173:])
train_data_tf_idf = s1_train_data_tf_idf + s2_train_data_tf_idf
valid_data_tf_idf = s1_valid_data_tf_idf + s2_valid_data_tf_idf
test_data_tf_idf = s1_test_data_tf_idf + s2_test_data_tf_idf

In [7]:
from sklearn.decomposition import TruncatedSVD
t_svd = TruncatedSVD(n_components=100)
train_data = t_svd.fit_transform(train_data_tf_idf)
valid_data = t_svd.transform(valid_data_tf_idf)
test_data = t_svd.transform(test_data_tf_idf)

In [16]:

import tensorflow, numpy as np
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras import metrics
model = Sequential()
ea = EarlyStopping(patience=10)
model.add(Dense(256, activation = "relu", input_shape=(100,)))
model.add(Dropout(0.4))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(4, activation = "softmax"))

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score
from functools import partial
def macro_f1_score(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=1)
    y_pred = tf.argmax(y_pred, axis=1)

    def f1_score_wrapper(y_true_np, y_pred_np):
        return f1_score(y_true_np, y_pred_np, average='macro')

    return tf.py_function(f1_score_wrapper, (y_true, y_pred), tf.float32)

rlr = keras.callbacks.ReduceLROnPlateau(factor=0.01, patience=10, min_lr=0.00001)

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = [macro_f1_score])

model.fit(np.array(train_data), train_onehot_encoded, validation_data = (np.array(valid_data), valid_onehot_encoded)
          , epochs = 25, callbacks=[ea])

Epoch 1/25
1817/1817 [==============================] - 6s 3ms/step - loss: 0.7962 - macro_f1_score: 0.4146 - val_loss: 0.7585 - val_macro_f1_score: 0.4660
Epoch 2/25
1817/1817 [==============================] - 6s 3ms/step - loss: 0.7529 - macro_f1_score: 0.4345 - val_loss: 0.7739 - val_macro_f1_score: 0.4611
Epoch 3/25
1817/1817 [==============================] - 6s 3ms/step - loss: 0.7405 - macro_f1_score: 0.4411 - val_loss: 0.7781 - val_macro_f1_score: 0.4677
Epoch 4/25
1817/1817 [==============================] - 6s 3ms/step - loss: 0.7307 - macro_f1_score: 0.4473 - val_loss: 0.7709 - val_macro_f1_score: 0.4682
Epoch 5/25
1817/1817 [==============================] - 6s 3ms/step - loss: 0.7243 - macro_f1_score: 0.4454 - val_loss: 0.7428 - val_macro_f1_score: 0.4729
Epoch 6/25
1817/1817 [==============================] - 5s 3ms/step - loss: 0.7194 - macro_f1_score: 0.4477 - val_loss: 0.7453 - val_macro_f1_score: 0.4673
Epoch 7/25
1817/1817 [==============================] - 5s 3ms/s

In [18]:
from sklearn.metrics import f1_score, balanced_accuracy_score
predicted = model.predict(np.array(valid_data))
predicted
valid_preds = [np.argmax(line) for line in predicted]
f1 = round(f1_score(valid_y, valid_preds, average='macro'), 2)
balanced_accuracy = round(100*balanced_accuracy_score(valid_y, valid_preds), 2)
print(f1)
print(balanced_accuracy)

96/96 [==============================] - 0s 1ms/step
0.35
36.27


## DL2

In [11]:
from tensorflow import keras
from tensorflow.keras import layers

In [27]:
#function created for training a neural network
def training_model(X_train, y_train, X_val, y_val, patience=10, opt = 'Adam', model='m3',min_lr=0.000001, lr=0.0001, factor = 0.01, epoch=10, batch=32, reduceLR=True):
    #initialize the reduceLR
    rlr = keras.callbacks.ReduceLROnPlateau(factor=factor, patience=patience, min_lr=min_lr)
    
    #Adam optimizer declared 
    if opt == 'Adam':
        optimizer = keras.optimizers.Adam(learning_rate=lr)
    
    #architecture 1
    if model == 'm3':
        model = keras.Sequential([
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(200, activation='relu'),
        layers.Dense(4)
    ])
    #arhitecture 2
    elif model == 'm2':
        model = keras.Sequential([
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.2),    
        layers.Dense(4)
    ])
    #arhitecture 3
    elif model == 'm1':
        model = keras.Sequential([
        layers.Dense(200, activation='relu'),
        layers.Dense(4)
    ])
    
    #compline de model using MAE loss and the Adam optimizer
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer)
    
    #use the reduceLR if needed
    if reduceLR:
        history = model.fit(X_train, y_train, batch_size=batch, epochs=epoch, validation_data=(X_val, y_val), callbacks=[rlr],)
    else:
        history = model.fit(X_train, y_train, batch_size=batch, epochs=epoch, validation_data=(X_val, y_val),)
    
    #return the history, predicted prob and the model
    return history, model.predict(X_val), model

In [13]:
train_y = df.iloc[:58114]['label']
valid_y = df.iloc[58114:61173]['label']

In [14]:

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(train_y)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
train_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


integer_encoded = label_encoder.fit_transform(valid_y)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
valid_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


[3 3 3 ... 3 2 3]


In [15]:
from sklearn.decomposition import TruncatedSVD

In [18]:
t_svd = TruncatedSVD(n_components=100)
train_data = t_svd.fit_transform(train_data_tf_idf)
valid_data = t_svd.transform(valid_data_tf_idf)
test_data = t_svd.transform(test_data_tf_idf)

In [ ]:
train_data[0]

In [39]:
train_data

array([[ 0.03740778, -0.00259994],
       [ 0.03414311, -0.00115177],
       [ 0.04892641, -0.00491652],
       ...,
       [ 0.04894245, -0.00391582],
       [ 0.06584624, -0.00721977],
       [ 0.04468842, -0.00105712]])

In [20]:
import numpy as np
np.array(train_data)

array([[ 0.03740777, -0.00259829, -0.00313735, ..., -0.00013723,
         0.0077212 , -0.00690627],
       [ 0.03414315, -0.00116434, -0.00108119, ..., -0.00963986,
        -0.00607675,  0.00315958],
       [ 0.04892642, -0.00491512, -0.00286651, ...,  0.00523675,
        -0.01081764,  0.00032247],
       ...,
       [ 0.04894245, -0.00391167, -0.00348638, ...,  0.01320282,
         0.00275113, -0.00511491],
       [ 0.0658463 , -0.00723348,  0.00294911, ...,  0.00437263,
        -0.00509896, -0.0014987 ],
       [ 0.0446884 , -0.00105094, -0.00297152, ..., -0.02006744,
        -0.00268463, -0.00984102]])

In [31]:
history, predicted, model = training_model(np.array(train_data) , train_onehot_encoded, np.array(valid_data), valid_onehot_encoded, patience=5,lr=0.0001, reduceLR=True, epoch=10, model='m2')

Epoch 1/10
1817/1817 [==============================] - 4s 2ms/step - loss: 6.3234 - val_loss: 5.6545 - lr: 1.0000e-04
Epoch 2/10
1817/1817 [==============================] - 4s 2ms/step - loss: 4.7353 - val_loss: 3.1358 - lr: 1.0000e-04
Epoch 3/10
1817/1817 [==============================] - 4s 2ms/step - loss: 4.1665 - val_loss: 2.0345 - lr: 1.0000e-04
Epoch 4/10
1817/1817 [==============================] - 3s 2ms/step - loss: 1.6348 - val_loss: 4.0265 - lr: 1.0000e-04
Epoch 5/10
1817/1817 [==============================] - 3s 2ms/step - loss: 8.5682 - val_loss: 9.1353 - lr: 1.0000e-04
Epoch 6/10
1817/1817 [==============================] - 3s 2ms/step - loss: 7.1669 - val_loss: 3.4944 - lr: 1.0000e-04
Epoch 7/10
1817/1817 [==============================] - 3s 2ms/step - loss: 5.4931 - val_loss: 11.0718 - lr: 1.0000e-04
Epoch 8/10
1817/1817 [==============================] - 3s 2ms/step - loss: 6.2664 - val_loss: 1.6271 - lr: 1.0000e-04
Epoch 9/10
1817/1817 [=========================

In [32]:
from sklearn.metrics import f1_score
predicted = model.predict(np.array(valid_data))
predicted
valid_preds = [np.argmax(line) for line in predicted]
f1 = round(f1_score(valid_y, valid_preds, average='macro'), 2)
print(f1)

96/96 [==============================] - 0s 1ms/step
0.03


In [43]:
import numpy as np
predicted = model.predict(valid_features)
predicted
test_preds = [np.argmax(line) for line in predicted]
test_guids = df_test['guid']
    

96/96 [==============================] - 0s 1ms/step


In [44]:
import csv
fields = ['guid', 'label']
rows = [ [test_guids.iloc[index], int(test_preds[index])] for index in range(len(test_guids))]

filename = "CNN_submission.csv"

with open(filename, 'w') as csvfile:   
    csvwriter = csv.writer(csvfile)  
        
    csvwriter.writerow(fields)  
         
    csvwriter.writerows(rows) 